# Stationary 3D Klein-Gordon soliton with cosine potential (aka the "Sine-Gordon" equation)

Now we will move from the non-relativistic Schrodinger equation to the fully relativistic Klein-Gordon equation. The KG Lagrangian for a real scalar field $\phi$ is given by
$$
\mathcal{L} = \frac{1}{2} \partial_\mu \phi \partial^\mu \phi - V(\phi).
$$
and we will focus specifically on the potential $V(\phi) = m^2 M^2 (1 - \cos(\phi/M))$

## Stationary solution

We can attempt a variational solution just like we did before. However, we first need to derive the Hamiltonian density, which is related to the Lagrangian by
$$
\mathcal{H} = \frac{\partial\mathcal{L}}{\partial\dot{\phi}} \dot{\phi} - \mathcal{L} = \frac{1}{2} \dot{\phi}^2 + \frac{1}{2} (\nabla \phi)^2 + V(\phi)
$$
The energy of any given field configuration is then simply the spatial integral over the Hamiltonian density, $E = \int d^3x\, \mathcal{H}$.

Now, unfortunately, the Klein-Gordon Lagrangian with cosine potential does not have a continuous symmetry that would lead to a conserved charge or particle number. However, it does have an _approximate_ symmetry on long time scales. To illustrate this consider the small-amplitude regime $\phi/M \ll 1$. In this regime, $V(\phi) \approx \frac{1}{2} m^2 \phi^2 - \frac{1}{4!} \frac{m^2}{M^2} \phi^4$. If we assume that we can decompose the wave function into a spatial piece and temporal piece like $\phi(x,t) = \phi(x)\cos(\omega t)$, then the Hamiltonian becomes
$$
\mathcal{H} = \frac{1}{2} \left[(\nabla \phi)^2 + (m^2 + \omega^2) \phi^2\right] \cos^2(\omega t) - \frac{1}{24}\frac{m^2}{M^2} \phi^4 \cos^4(\omega t)
$$
Since this is periodic, we can average over time, allowing us to replace $\cos^2(\omega t) \rightarrow 1/2$ and $\cos^4(\omega t) \rightarrow 3/8$. We could do the same thing to higher-order terms as well, but we will truncate at $O(\phi^4)$ for now. Once again, we need to assume a trial wave function with some free parameters we can tune to minimize the energy. The ground state wave function should be spherically symmetric, so we will first assume a Gaussian ansatz:

In [86]:
import sympy as sp

phi = sp.Function("\phi", real=True)
r, R, phi0 = sp.symbols("r R \phi_0", positive=True)

def trial_phi(r):
    return phi0*sp.exp(-(r/R)**2)

sp.Eq(phi(r), trial_phi(r))

Eq(\phi(r), \phi_0*exp(-r**2/R**2))

In [187]:
N, w = sp.symbols("N, \omega", positive=True)
sol = sp.solveset(sp.Eq(N, 4*sp.pi*w*sp.integrate((r**2)*(trial_phi(r)**2), (r, 0, sp.oo))), phi0)
sp.Eq(phi0, sol.args[0])

Eq(\phi_0, 2**(3/4)*sqrt(N)/(pi**(3/4)*R**(3/2)*sqrt(\omega)))

Then we plug this into the Hamiltonian density and integrate to get the variational energy

In [188]:
m, w, M = sp.symbols("m \omega M", positive=True)
H_density = (sp.Rational(1,4)*(sp.diff(phi(r), r)**2 + (m**2 + w**2)*phi(r)**2) - sp.Rational(3,8)*(m/M)**2*phi(r)**4).subs(phi(r), trial_phi(r)).doit()
H = (4*sp.pi*sp.integrate((r**2)*H_density, (r, 0, sp.oo)).subs(phi0, sol.args[0])).simplify().collect(R)
H

N*\omega/4 + N*m**2/(4*\omega) + 3*N/(4*R**2*\omega) - 3*N**2*m**2/(8*pi**(3/2)*M**2*R**3*\omega**2)

And finally, we minimize the energy with respect to $R$ and $\omega$

In [197]:
sol = sp.solve([
        sp.diff(H, R).simplify(),
        sp.diff(H, w).simplify()
], (R, w))
sp.FiniteSet(*sol)

FiniteSet((sqrt(16*pi**3*M**4 + 9*N**2*m**4)/(4*pi**(3/2)*M**2*m), 3*N*m**3/sqrt(16*pi**3*M**4 + 9*N**2*m**4)))

These can be simplified a bit:
$$
R = \frac{1}{m} \left[ 1 + \widetilde{N}^2 \right]^{1/2}, \quad
\omega = m \left[ 1 + \widetilde{N}^{-2} \right]^{-1/2}, \quad\text{where}\quad
\widetilde{N} = \frac{3 m^2}{4\pi^{3/2} M^2} N
$$
notice how $\omega \le m$ for any value of $N$, and the equality is saturated for $N \rightarrow \infty$.

## Numerical computations

Like we have done vefore, we can easily non-dimensionalize the Lagrangian by making the substitutions $\phi \rightarrow M\theta$, $\vec{x} \rightarrow \vec{\xi}/m$, and $t \rightarrow \tau/m$, resulting in a much simpler expression with zero free parameters:
$$
\mathcal{L} \rightarrow (mM)^2 \tilde{\mathcal{L}} = (mM)^2 \left[ \frac{1}{2} \left(\frac{d\theta}{d\tau}\right)^2 - \frac{1}{2} (\nabla_\xi \theta)^2 - \left( 1 - \cos\theta \right) \right]
$$
The corresponding Hamiltonian density is given by
$$
\tilde{\mathcal{H}} = \frac{1}{2} \left(\frac{d\theta}{d\tau}\right)^2 + \frac{1}{2} (\nabla_\xi \theta)^2 + \left( 1 - \cos\theta \right)
$$

If we separate the oscillatory component of the field from the spatial field value via $\theta = \theta \cos(\omega\tau/m)$, we can then take the time average of the Hamiltonian density over a single period to get
$$
\langle \tilde{\mathcal{H}} \rangle = \frac{1}{4} \theta^2 + \frac{1}{4} (\nabla_\xi \theta)^2 + \int_0^{2\pi} \frac{d\theta^\prime}{2\pi} \left( 1 - \cos\left( \theta \cos \theta^\prime \right)\right)
$$

Discretizing this on a 1D grid (along the radial component of $\vec{\xi}$; the soliton is actually 3D), we arrive at
$$
\langle\tilde{\mathcal{H}}\rangle \approx \pi \sum_{r^\prime} \delta r\, \left(\frac{\phi_{r^\prime + 1} - \phi_{r^\prime}}{\delta r}\right)^2 + 
$$

We can derive the equations of motion from the Lagrangian using the Euler-Lagrange equations, which leads us to
$$
\partial^2 \phi - V^\prime(\phi) = 0
$$
If we assume a "cosine potential" of the form $V(\phi) = m^2 M^2 (1 - \cos(\phi/M))$, then this becomes
$$
\ddot{\phi} - \nabla^2 \phi + m^2 M \sin(\phi/M) = 0
$$

Like we have done before, we can easily non-dimensionalize this equation by making the substitutions $\phi \rightarrow M\theta$, $\vec{x} \rightarrow \vec{\xi}/m$, and $t \rightarrow \tau/m$, resulting in a much simpler equation with zero free parameters:
$$
\frac{\partial^2\theta}{\partial\tau^2} - \nabla_\xi^2 \theta + \sin\theta = 0
$$